### 環境

In [25]:
import requests
import json
import re
from bs4 import BeautifulSoup as bs

### 取得QA資料

In [22]:
page = 1
pageSize = 218
url = f"https://www.water.gov.tw/ch/Subject/SwitchPage?listType=1&nodeId=4889&page={page}&pageSize={pageSize}&SearchCategory=-1"

headers = {
    'accept-language': 'zh-TW,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'origin': 'https://www.water.gov.tw',
    'referer': 'https://www.water.gov.tw/ch/Subject?nodeId=4889',
    'sec-ch-ua': '"Microsoft Edge";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
    'x-requested-with': 'XMLHttpRequest',
    'cookie': '__RequestVerificationToken=8QFguW-WEAaQaID9nf9_Bg304GqUUXgnginLua-ygHFvtuFJbb3BmhRFZRHpqj_YWO8tH14HVOUNiQim8pnxbH3OryQlYuVRgjHS88-VYPI1; _ga=GA1.1.425123601.1746197373; TS012f1a86=0147fc99a89444c2651d79ea52449630b4b418a0df8ba8c30a4fbd48159f018ea7be5198f6426226fca8564e984ead4d499a20490b93e4b46bda934f9b51b11cbc4f4afa33; _ga_87J75GBNX4=GS1.1.1746197373.1.1.1746197997.0.0.0; TSb55b7e92027=08f1dc42d4ab2000395ecf9ccb4f59358e3bfae9e87326f4502899aa68531d0eda8f258b80d89d0708bde5cc8a113000d808c93986cbb27a81da31c5244747fbf8d687376e004413750e9108034711d768937560bc91549ea679f7b2d4fabfc6',
}

data = {
    'X-Requested-With': 'XMLHttpRequest',
}

response = requests.post(
    url,
    headers=headers,
    data=data,
)

In [ ]:
# print(response.text)

soup = bs(response.text, 'html.parser')

# 尋找所有公告項目
bulletin_items = soup.select('.bulletin_list_item')

# 準備輸出資料
result = {"data": []}

# 遍歷每個公告項目並提取資訊
for item in bulletin_items:
    link = item.find('a')
    href = "https://www.water.gov.tw" + link.get('href') if link else ""
    title = link.get('title') if link else ""
    date = item.select_one('.bulletin_list_date').text.strip() if item.select_one('.bulletin_list_date') else ""
    
    # 將資料添加到結果中
    result["data"].append({
        "href": href,
        "title": title,
        "date": date
    })

# 輸出JSON格式
print(json.dumps(result, ensure_ascii=False, indent=4))

# 存檔
with open('water_data.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

# 讀取
# with open('water.json', 'r', encoding='utf-8') as f:
#     result = json.load(f)

### 取得QA內容

In [23]:
# 開啟檔案
# with open('water_data.json', 'r', encoding='utf-8') as f:
#     result = json.load(f)

# print(result["data"][0])


headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'zh-TW,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'max-age=0',
    'priority': 'u=0, i',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0',
    'cookie': '__RequestVerificationToken=8QFguW-WEAaQaID9nf9_Bg304GqUUXgnginLua-ygHFvtuFJbb3BmhRFZRHpqj_YWO8tH14HVOUNiQim8pnxbH3OryQlYuVRgjHS88-VYPI1; _ga=GA1.1.425123601.1746197373; TS012f1a86=0147fc99a82f25c52e93da44fc92a890280af23408c8784a74107278e22c550bbe33d65e098df6cdc36cd1b27adf2870ac817a315bea5dc530de25e9af9069b34919d1da19; _ga_87J75GBNX4=GS1.1.1746197373.1.1.1746200107.0.0.0; TSb55b7e92027=08f1dc42d4ab2000160597b2b55f2d4f7da62a3f9a1b45eeb8abff0ad70c1f69a58d15e65af999ef082de6fe3e1130007dc2977440f9497321c37ed5da3a16507c58513a677e63bfc299f772367363505ff7b261aead55bdacfea3bc4eac553f',
}

# response = requests.get(result["data"][0]["href"], headers=headers)
# response = requests.get("https://www.water.gov.tw/ch/Subject/Detail/3743?nodeId=4889", headers=headers)


In [18]:
# 處理 main_page_content 內容
def extract_main_page_content(soup):
    content_div = soup.select_one('.main_page_content')
    if not content_div:
        print("沒有main_page_content")
        return ""
    
    # 情況1: 有序列表
    if content_div.select('ol li'):
        print("類型：有序列表")
        items = content_div.select('ol li')
        result = []
        for i, item in enumerate(items, 1):
            result.append(f"{i}. {item.get_text(strip=True)}")
        return "\n".join(result)
    
    # 情況3: 純文字
    else:
        print("類型：純文字")
        # get_text(strip=True) 會移除所有 HTML 標籤，包括 <br>，而不會將 <br> 轉換為換行符
        # 手動處理 <br> 標籤
        text = ""
        for element in content_div.contents:
            if element.name == 'br':
                text += ''
            elif isinstance(element, str):
                text += element
            else:
                text += element.get_text(strip=True)
        return text.strip()

# 處理 main_page_office 內容
def extract_main_page_office(soup):
    office_div = soup.select_one('.main_page_office')
    if not office_div:
        return ""
    
    print("類型：表格")
    result = []
    
    # 處理表格內容
    rows = office_div.select('tr')
    for row in rows:
        th = row.select_one('th')
        td = row.select_one('td')
        
        if th and td:
            header = th.get_text(strip=True)
            
            # 檢查是否有多個段落
            paragraphs = td.select('p')
            if paragraphs:
                value = "\n".join([p.get_text(strip=True) for p in paragraphs])
            else:
                value = td.get_text(strip=True)
            
            result.append(f"{header} {value}")
    
    return "\n".join(result)

In [ ]:
log_txt = ""
result_content = result["data"]

for i in result_content:
    response = requests.get(i["href"], headers=headers)
    response.encoding = 'utf-8'

    soup = bs(response.text, 'html.parser')

    try:
        # 提取頁面內容
        page_content = extract_main_page_office(soup)
        if not page_content:
            page_content = extract_main_page_content(soup)
    except Exception as e:
        print(f"提取頁面內容發生錯誤，{result_content.index(i)}，{e}")
        log_txt += f"{result_content.index(i)} 錯誤\n"

    if page_content == "":
        print(result_content.index(i), "錯誤")
        log_txt += f"{result_content.index(i)} 錯誤\n"

    # 提取頁面資訊
    page_info_view = soup.select_one('.main_page_info_view')
    page_info_view = page_info_view.get_text(strip=True) if page_info_view else ""

    page_info_update = soup.select_one('.main_page_info_update')
    page_info_update = page_info_update.get_text(strip=True) if page_info_update else ""

    # 加入 i 的字典
    i["page_content"] = page_content
    i["page_info_view"] = page_info_view
    i["page_info_update"] = page_info_update

    if result_content.index(i+1) % 10 == 0:
        print(result_content.index(i+1),"已經完成")

    # print("\n頁面內容:")
    # print(page_content)

    # print("\n頁面瀏覽資訊:")
    # print(page_info_view)
    # print("\n頁面更新資訊:")
    # print(page_info_update)

print("Done")

In [20]:
# 存檔
with open('water_data_v1.json', 'w', encoding='utf-8') as f:
    json.dump(result_content, f, ensure_ascii=False, indent=4)

### 抓 File(PDF)

In [24]:
with open('water_data_content_v1.json', 'r', encoding='utf-8') as f:
    result = json.load(f)

In [37]:
# 處理 main_page_content 內容
def extract_file(soup):
    # 尋找 class="alink" 的 div
    alink_div = soup.select_one('.alink')
    
    # 如果找到 alink_div
    if alink_div:
        # 找到第一個 a 標籤，這應該是 PDF 連結
        pdf_link = alink_div.select_one('a')
        
        if pdf_link and pdf_link.has_attr('href'):
            # 取得相對路徑
            pdf_relative_url = pdf_link['href']
            # 組合成完整 URL
            pdf_url = f"https://www.water.gov.tw{pdf_relative_url}"
            return pdf_url
    
    # 如果沒有找到 PDF 連結，返回 None
    return False

In [49]:
count = 0
index = 0

for i in result:

    response = requests.get(i["href"], headers=headers)
    response.encoding = 'utf-8'

    soup = bs(response.text, 'html.parser')

    try:
        # 提取頁面內容
        url_pdf = extract_file(soup)
        if url_pdf:
            count += 1
            print(index, "有檔案", url_pdf)
        else:
            # print(index,"沒有PDF")
            url_pdf = ""

    except Exception as e:
        print(f"提取頁面內容發生錯誤，{result[0]}，{e}")


    # # 加入 i 的字典
    i["page_file"] = url_pdf
    
    index += 1

    if index % 10 == 0:
        print(index, "已經完成")


print("總共有",count,"個PDF")

10 已經完成
13 有PDF https://www.water.gov.tw/ch/ServerFile/Get/70355d26-a12a-43f4-acdb-0942f508e865?nodeId=4889
20 已經完成
20 有PDF https://www.water.gov.tw/ch/ServerFile/Get/3a92a6fc-e1f7-4b47-91dc-6f45dc8700e2?nodeId=4889
21 有PDF https://www.water.gov.tw/ch/ServerFile/Get/1854dce9-6f1e-4d13-9558-22c464d4a148?nodeId=4889
30 已經完成
40 已經完成
50 已經完成
60 已經完成
70 已經完成
80 已經完成
90 已經完成
100 已經完成
110 已經完成
120 已經完成
130 已經完成
140 已經完成
150 已經完成
160 已經完成
170 已經完成
180 已經完成
183 有PDF https://www.water.gov.tw/ch/ServerFile/Get/af5322f3-4fca-4872-9b9c-a6e68eef09bf?nodeId=4889
190 已經完成
200 已經完成
202 有PDF https://www.water.gov.tw/ch/ServerFile/Get/a5399a5a-99b7-4a8e-88a3-fc3f87afb79a?nodeId=4889
208 有PDF https://www.water.gov.tw/ch/ServerFile/Get/07a111bb-ed90-4fe9-9fcc-faf3c3f166de?nodeId=4889
210 已經完成
總共有 6 個PDF


In [53]:
# 存檔
with open('water_data_content_v2.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

### 分類

In [9]:
with open('water_data_content_v2.json', 'r', encoding='utf-8') as f:
    result = json.load(f)

In [ ]:
index = 0
for i in result:
    print("index:", index)
    print("標題：", i["title"])
    print("內容：", i["page_content"])
    print("---")
    index += 1

In [7]:
lst = ["電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "用戶帳戶與用水設備管理", "帳單與繳費管理", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "帳單與繳費管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "電子帳單、簡訊帳單及通知服務", "用戶帳戶與用水設備管理", "污水下水道與污水使用費", "污水下水道與污水使用費", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "水質、淨水與生活應用", "帳單與繳費管理", "帳單與繳費管理", "帳單與繳費管理", "帳單與繳費管理", "帳單與繳費管理", "帳單與繳費管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "水價政策與事業經營", "水價政策與事業經營", "水價政策與事業經營", "污水下水道與污水使用費", "污水下水道與污水使用費", "污水下水道與污水使用費", "污水下水道與污水使用費", "污水下水道與污水使用費", "污水下水道與污水使用費", "污水下水道與污水使用費", "污水下水道與污水使用費", "污水下水道與污水使用費", "污水下水道與污水使用費", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "電子帳單、簡訊帳單及通知服務", "緊急停水、計畫停水與應變", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "水價政策與事業經營", "水價政策與事業經營", "水價政策與事業經營", "帳單與繳費管理", "帳單與繳費管理", "用戶帳戶與用水設備管理", "帳單與繳費管理", "帳單與繳費管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "帳單與繳費管理", "帳單與繳費管理", "帳單與繳費管理", "帳單與繳費管理", "水價政策與事業經營", "帳單與繳費管理", "帳單與繳費管理", "帳單與繳費管理", "帳單與繳費管理", "水價政策與事業經營", "帳單與繳費管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "用戶帳戶與用水設備管理", "App／網站使用與隱私政策", "用戶帳戶與用水設備管理", "水質、淨水與生活應用", "水質、淨水與生活應用", "水價政策與事業經營", "水價政策與事業經營", "水價政策與事業經營", "水價政策與事業經營", "水價政策與事業經營", "水價政策與事業經營", "水價政策與事業經營", "水價政策與事業經營", "水價政策與事業經營", "App／網站使用與隱私政策", "水價政策與事業經營", "App／網站使用與隱私政策", "水價政策與事業經營"]

In [10]:
index = 0

for i in result:
    # 加入 i 的字典
    i["Category"] = lst[index]
    
    index += 1

    if index % 10 == 0:
        print(index, "已經完成")

print("Done")

10 已經完成
20 已經完成
30 已經完成
40 已經完成
50 已經完成
60 已經完成
70 已經完成
80 已經完成
90 已經完成
100 已經完成
110 已經完成
120 已經完成
130 已經完成
140 已經完成
150 已經完成
160 已經完成
170 已經完成
180 已經完成
190 已經完成
200 已經完成
210 已經完成
Done


In [11]:
# 存檔
with open('water_data_content_v3.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

## 抓營業所資料

### 定義標頭

In [14]:
headers = {
    'accept': '*/*',
    'accept-language': 'zh-TW,zh;q=0.9',
    'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'origin': 'https://www.water.gov.tw',
    'priority': 'u=1, i',
    'referer': 'https://www.water.gov.tw/ch/Location?nodeId=4890',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36 Edg/136.0.0.0',
    'x-requested-with': 'XMLHttpRequest',
}

data = {
    'X-Requested-With': 'XMLHttpRequest',
}

### 定義函數

In [36]:
def extract_info_from_html_v0(html_content, config=None):
    """
    從 HTML 中提取資訊並轉換為 JSON 格式
    
    Args:
        html_content (str): HTML 內容
        config (dict): 自定義提取配置，包含要提取的元素選擇器
    
    Returns:
        str: JSON 格式的提取結果
    """
    soup = bs(html_content, 'html.parser')
    data = []
    
    # 預設配置 - 基於台水服務所的結構
    if config is None:
        config = {
            'container_selector': 'div.bulletin_list_item',
            'fields': {
                'title': {'selector': 'div.bulletin_list_title', 'attribute': 'text'},
                'phone': {'selector': 'p.phone', 'attribute': 'text'},
                'address': {'selector': 'p.map', 'attribute': 'text'},
                'region': {'selector': 'div.bulletin_list_region', 'attribute': 'text'},
                'code': {'selector': 'p:-soup-contains("代碼")', 'attribute': 'text'},
                'area': {'selector': 'p:-soup-contains("轄區")', 'attribute': 'text'}
            }
        }
    
    # 找到所有容器元素
    items = soup.select(config['container_selector'])
    
    for item in items:
        item_data = {}
        
        for field_name, field_config in config['fields'].items():
            element = item.select_one(field_config['selector'])
            
            if element:
                if field_config['attribute'] == 'text':
                    item_data[field_name] = element.get_text(strip=True)
                else:
                    item_data[field_name] = element.get(field_config['attribute'], '')
            else:
                item_data[field_name] = ''
        
        if any(item_data.values()):  # 只添加非空的項目
            data.append(item_data)
    
    return json.dumps(data, ensure_ascii=False, indent=2)

def save_to_json_file(data, filename):
    """
    將資料存成 JSON 檔案
    
    Args:
        data (str): JSON 字串
        filename (str): 檔案名稱
    """
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(data)
    print(f"資料已存儲到 {filename}")

def extract_info_from_html(html):
    """
    從HTML中提取服務所資訊並轉換為JSON格式
    
    Args:
        html (str): 包含服務所資訊的HTML字符串
        
    Returns:
        str: JSON格式的字符串
    """
    soup = bs(html, 'html.parser')
    result = []
    
    # 找到所有服務所項目
    items = soup.select('.bulletin_list_item')
    
    for item in items:
        # 提取標題
        title = item.select_one('.bulletin_list_title').get_text(strip=True) if item.select_one('.bulletin_list_title') else ''
        if "管理處" in title:
            continue
        
        # 提取電話
        phone = item.select_one('.org_contact .phone').get_text(strip=True) if item.select_one('.org_contact .phone') else ''
        
        # 提取地址
        address = item.select_one('.org_contact .map').get_text(strip=True) if item.select_one('.org_contact .map') else ''
        
        # 提取區域
        region = item.select_one('.bulletin_list_region').get_text(strip=True) if item.select_one('.bulletin_list_region') else ''
        
        # 初始化代碼和轄區
        code = ''
        area = ''
        
        # 從org_contact段落中提取代碼和轄區
        org_contact_ps = item.select('.org_contact p')
        for p in org_contact_ps:
            text = p.get_text(strip=True)
            if text.startswith('代碼:'):
                code = text
            elif text.startswith('轄區:'):
                area = text
        
        # 提取詳細頁面連結
        href = ''
        a_tag = item.find('a')
        if a_tag and a_tag.has_attr('href'):
            href = 'https://www.water.gov.tw' + a_tag['href']
        
        # 提取地圖URL
        mapURL = ''
        map_div = item.select_one('.org_contact .map')
        if map_div and map_div.has_attr('onclick'):
            onclick_value = map_div['onclick']
            # 使用正則表達式提取window.open中的URL
            match = re.search(r"window.open\('(.*?)'\)", onclick_value)
            if match:
                mapURL = match.group(1)
        
        # 組合結果
        result.append({
            'title': title,
            'phone': phone,
            'address': address,
            'region': region,
            'code': code,
            'area': area,
            'href': href,
            'mapURL': mapURL
        })
    
    # return json.dumps(result, ensure_ascii=False, indent=2)
    return result

### 主程式

In [38]:
# 限定 服務所、營運所，排除管理處

page = 1
pageSize = 128

URL = f"https://www.water.gov.tw/ch/Location/SwitchPage?listType=1&categoryType=3&nodeId=4890&page={page}&pageSize={pageSize}"

response = requests.post(URL, headers=headers, data=data)
response.encoding = 'utf-8'

# soup = bs(response.text, 'html.parser')

location_info_data = extract_info_from_html(response.text)

# 存檔
with open("water_location_data_v1.json", "w", encoding="utf-8") as f:
# ensure_ascii 是否強制使用 ASCII 字符
    json.dump(location_info_data, f, ensure_ascii=False, indent=2)
